In [1]:
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import albumentations as A
from pathlib import Path
import os
from tqdm import tqdm
import sklearn.metrics
import PIL.Image as Image, PIL.ImageDraw as ImageDraw, PIL.ImageFont as ImageFont
from matplotlib.font_manager import FontProperties
prop = FontProperties()
plt.style.use('seaborn-dark-palette')
prop.set_file('../data/kalpurush.ttf')
%matplotlib inline

In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from IPython.display import SVG

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D, AvgPool2D

In [4]:
train_df = pd.read_csv('/home/sayan/Documents/Bengali_Grapheme/data/train.csv')
strs = train_df['image_id'].values
strs = [string+'_f.png' for string in strs]
train_df.drop(['image_id'],axis=1,inplace=True)
train_df['image_id'] = strs

In [5]:
class_map = pd.read_csv('/home/sayan/Documents/Bengali_Grapheme/data/class_map.csv')

In [6]:
def apply_augmentation(image):
    augmentation_pipeline = A.Compose(
        [
            A.OneOf(
                [
                    # apply one of transforms to 50% of images
                    A.RandomContrast(), # apply random contrast
                    A.RandomGamma(), # apply random gamma
                    A.RandomBrightness(), # apply random brightness
                ],
                p = 0.5
            ),
            A.OneOf(
                [
                    # apply one of transforms to 50% images
                    A.ElasticTransform(alpha = 120,sigma = 120 * 0.05,alpha_affine = 120 * 0.03),
                    A.GridDistortion(),
                    A.OpticalDistortion(distort_limit = 2,shift_limit = 0.5),
                ],
                p = 0.5
            )
        ],
        p = 0.5
    )
    images_aug = augmentation_pipeline(image = image)['image']
    return images_aug

In [7]:
def modified_recall(y_true, y_pred):
    
    scores = []
    for component in ['grapheme_root', 'consonant_diacritic', 'vowel_diacritic']:
        y_true_subset = solution[solution[component] == component]['target'].values
        y_pred_subset = submission[submission[component] == component]['target'].values
        scores.append(sklearn.metrics.recall_score(
            y_true_subset, y_pred_subset, average='macro'))
    
    final_score = np.average(scores, weights=[2,1,1])

In [8]:
datadir_toy_processed_images = '/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images'
train_df_sample = pd.DataFrame()
for (directory, _ , image_names) in os.walk(datadir_toy_processed_images):
        train_df_sample = train_df[train_df['image_id'].isin(image_names)]

### Train Test Split

In [9]:
from sklearn.model_selection import train_test_split

train_df_1, valid_df_1 = train_test_split(train_df_sample,stratify=train_df_sample[['grapheme_root',
                                                                                'vowel_diacritic',
                                                                                'consonant_diacritic']])

#train_df_1, valid_df_1 = train_test_split(train_df_sample)

In [10]:
train_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255,
                                                               preprocessing_function=apply_augmentation)

valid_im_generator = tf.keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)

train_gen_image = train_im_generator.flow_from_dataframe(train_df_1,
                                                   directory='/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images',
                                                   x_col='image_id',
                                                   y_col=['grapheme_root','vowel_diacritic','consonant_diacritic'],
                                                   target_size = (64,64),
                                                   class_mode = 'multi_output',
                                                   color_mode = 'grayscale',
                                                   shuffle=False, 
                                                   batch_size = 10,
                                                   seed=42)

valid_gen_image = valid_im_generator.flow_from_dataframe(valid_df_1,
                                                   directory='/home/sayan/Documents/Bengali_Grapheme/images/toy_processed_images',
                                                   x_col='image_id',
                                                   y_col=['grapheme_root','vowel_diacritic','consonant_diacritic'],
                                                   target_size = (64,64),
                                                   class_mode = 'multi_output',
                                                   color_mode = 'grayscale',
                                                   shuffle=False, 
                                                   batch_size = 10,
                                                   seed=42)

Found 15063 validated image filenames.
Found 5021 validated image filenames.


In [11]:
def oneHotEncode_outputs(gen_image):
    while True:
        output = next(gen_image)
        img = output[0]
        labels = output[1]
        #grapheme_label = np.zeros(168)
        #vowel_label = np.zeros(11)
        #consonant_label = np.zeros(7)
        for i in range(3):
            if i == 0:
                a = labels[i]
                b = np.zeros((a.size, 168))
                b[np.arange(a.size),a] = 1
                labels[i] = b
            elif i == 1:
                a = labels[i]
                b = np.zeros((a.size, 11))
                b[np.arange(a.size),a] = 1
                labels[i] = b
            else:
                a = labels[i]
                b = np.zeros((a.size, 7))
                b[np.arange(a.size),a] = 1
                labels[i] = b
        yield (img, labels)

### Check if generator labels are accurate

In [12]:
def image_from_char(char):
    HEIGHT = 236
    WIDTH = 236
    image = Image.new('RGB', (WIDTH, HEIGHT))
    draw = ImageDraw.Draw(image)
    myfont = ImageFont.truetype('../data/kalpurush.ttf', 120)
    w, h = draw.textsize(char, font=myfont)
    draw.text(((WIDTH - w) / 2,(HEIGHT - h) / 3), char, font=myfont)

    return image

In [13]:
def check_generators(gen_image, train_df,fname,class_map=class_map):
    
    output = next(gen_image)
    fig, axs = plt.subplots(gen_image.batch_size,4, figsize=(64, 64))
    gs1 = gridspec.GridSpec(gen_image.batch_size, 4)
    
    gs1.update(wspace=0.025, hspace=0.05)
    plt.subplots_adjust(wspace=-0.8, hspace=0.1)
    images = output[0]
    roots = output[1][0]
    vowels = output[1][1]
    consonants = output[1][2]
    
    for i in range(gen_image.batch_size): 
        image = images[i].reshape(64,64)
        grapheme_root = roots[i]
        vowel_diacritic = vowels[i]
        consonant_diacritic = consonants[i]
        vowel_label = class_map[(class_map['component_type'] == 'vowel_diacritic') 
                            & (class_map['label'] == vowel_diacritic)]['component'].values[0]
        consonant_label = class_map[(class_map['component_type'] == 'consonant_diacritic') 
                            & (class_map['label'] == consonant_diacritic)]['component'].values[0]
        root_label = class_map[(class_map['component_type'] == 'grapheme_root') 
                            & (class_map['label'] == grapheme_root)]['component'].values[0]
        
        axs[i,0].imshow(image,cmap='Greys')
        axs[i,0].axis('off')
        axs[i,0].set_aspect('equal')
        axs[i,1].imshow(image_from_char(root_label), cmap='Greys')
        axs[i,1].axis('off')
        axs[i,1].set_aspect('equal')
        axs[i,2].imshow(image_from_char(vowel_label), cmap='Greys')
        axs[i,2].axis('off')
        axs[i,2].set_aspect('equal')
        axs[i,3].imshow(image_from_char(consonant_label), cmap='Greys')
        axs[i,3].axis('off')
        axs[i,3].set_aspect('equal')
        
    fig.savefig('../results/'+fname+'.png')

In [ ]:
check_generators(valid_gen_image,valid_df_1,'validation_images',class_map);

---

### Creating Callbacks

In [16]:
class CyclicLR(keras.callbacks.Callback):
    
    def __init__(self,base_lr, max_lr, step_size, base_m, max_m, cyclical_momentum):
 
        self.base_lr = base_lr
        self.max_lr = max_lr
        self.base_m = base_m
        self.max_m = max_m
        self.cyclical_momentum = cyclical_momentum
        self.step_size = step_size
        
        self.clr_iterations = 0.
        self.cm_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}
        
    def clr(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)          
            return self.base_lr-(self.base_lr-self.base_lr/100)*np.maximum(0,(1-x))
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0,(1-x))
    
    def cm(self):
        
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        
        if cycle == 2:
            
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1) 
            return self.max_m
        
        else:
            x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
            return self.max_m - (self.max_m-self.base_m)*np.maximum(0,(1-x))
        
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())
            
        if self.cyclical_momentum == True:
            if self.clr_iterations == 0:
                K.set_value(self.model.optimizer.momentum, self.cm())
            else:
                K.set_value(self.model.optimizer.momentum, self.cm())
            
            
    def on_batch_begin(self, batch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)
        
        if self.cyclical_momentum == True:
            self.history.setdefault('momentum', []).append(K.get_value(self.model.optimizer.momentum))

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
        
        if self.cyclical_momentum == True:
            K.set_value(self.model.optimizer.momentum, self.cm())

batch_size = train_gen_image.batch_size
epochs = 10
max_lr = 0.5
base_lr = max_lr/10
max_m = 0.98
base_m = 0.85

cyclical_momentum = True
augment = True
cycles = 2.35

iterations = round(len(train_df)/batch_size*epochs)
iterations = list(range(0,iterations+1))
step_size = len(iterations)/(cycles)

In [17]:
checkpoint_cb = keras.callbacks.ModelCheckpoint('my_keras_vanilla.h5',save_best_only=True)

early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,restore_best_weights=True)

reduceLR_cb = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                                                patience=5, min_lr=0.001)

root_logdir = '/home/sayan/Documents/Bengali_Grapheme/logs/'

def get_run_logdir(name):
    import time
    run_id = time.strftime(name)
    return os.path.join(root_logdir,run_id)

run_logdir = get_run_logdir('alexnet-run_%Y_%m_%d-%H_%M_%S')

tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

cyclicLR_cb =  CyclicLR(base_lr=base_lr,
                max_lr=max_lr,
                step_size=step_size,
                max_m=max_m,
                base_m=base_m,
                cyclical_momentum=cyclical_momentum)

### Vanilla Model

In [ ]:
def model_vanilla():
    
    input_ = Input(shape=(64,64,1))
    conv1 = Conv2D(filters=6, kernel_size=(3, 3), activation='relu')(input_)
    avgpool1 = AvgPool2D()(conv1)
    
    conv2 = Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(avgpool1)
    avgpool2 = AvgPool2D()(conv2)

    flat = Flatten()(avgpool2)
    
    dense_root_1 = Dense(2000,activation='relu')(flat)
    dense_root_2 = Dense(1500,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    dense_root_4 = Dense(800,activation='relu')(dense_root_3)
    dense_root_5 = Dense(250,activation='relu')(dense_root_4)
    output_root = Dense(168,activation='softmax')(dense_root_4)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [ ]:
model_vanilla = model_vanilla()
keras.utils.plot_model(model_vanilla, '../results/vanilla_model.png', expand_nested=True, show_shapes=True);

In [ ]:
model_vanilla.compile(loss = keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

history_vanilla = model_vanilla.fit(train_gen_image,
                                    validation_data=valid_gen_image,
                                    epochs = 5,callbacks=[tensorboard_cb,reduceLR_cb,early_stopping_cb])

---

### Alex Net

In [18]:
def model_alexnet():
    
    input_ = Input(shape=(64,64,1))
    
    conv1 = Conv2D(filters=96, kernel_size=(11, 11), strides=4, padding='valid', activation='relu')(input_)
    maxpool1 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='valid')(conv1)
    
    conv2 = Conv2D(filters=256, kernel_size=(5, 5), strides=1, padding='same', activation='relu')(maxpool1)
    maxpool2 = MaxPooling2D(pool_size=(3, 3), strides=2, padding='valid')(conv2)
    
    conv3 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(maxpool2)
    conv4 = Conv2D(filters=384, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(conv3)
    conv5 = Conv2D(filters=256, kernel_size=(3, 3), strides=1, padding='same', activation='relu')(conv4)
    
    flat = Flatten()(conv5)
    
    dense_root_1 = Dense(4096,activation='relu')(flat)
    dense_root_2 = Dense(4096,activation='relu')(dense_root_1)
    dense_root_3 = Dense(1000,activation='relu')(dense_root_2)
    output_root = Dense(168,activation='softmax')(dense_root_3)
    
    dense_vowel_1 = Dense(800,activation='relu')(flat)
    dense_vowel_2 = Dense(600,activation='relu')(dense_vowel_1)
    dense_vowel_3 = Dense(100,activation='relu')(dense_vowel_2)
    output_vowel = Dense(11,activation='softmax')(dense_vowel_3)
    
    dense_consonant_1 = Dense(800,activation='relu')(flat)
    dense_consonant_2 = Dense(600,activation='relu')(dense_consonant_1)
    dense_consonant_3 = Dense(100,activation='relu')(dense_consonant_2)
    output_consonant = Dense(7,activation='softmax')(dense_consonant_3)
    
    model = keras.Model(inputs=[input_],outputs=[output_root,output_vowel,output_consonant])
    
    return model

In [19]:
model_alexnet = model_alexnet()
keras.utils.plot_model(model_alexnet, '../results/vanilla_alexnet.png', expand_nested=True, show_shapes=True);

In [ ]:
model_alexnet.compile(loss = keras.losses.sparse_categorical_crossentropy,
              optimizer = keras.optimizers.Adam(learning_rate=0.05),
              metrics=['accuracy'])

history_alexnet = model_alexnet.fit(train_gen_image,
                                    validation_data=valid_gen_image,
                                    epochs = 5,callbacks=[tensorboard_cb,reduceLR_cb,early_stopping_cb])

Train for 1507 steps, validate for 503 steps
Epoch 1/5
 985/1507 [==================>...........] - ETA: 1:46 - loss: 47862550.8511 - dense_3_loss: 47200724.0000 - dense_7_loss: 410341.0625 - dense_11_loss: 251470.5469 - dense_3_accuracy: 0.0252 - dense_7_accuracy: 0.1879 - dense_11_accuracy: 0.6228